## Script for creating training sample
### Johan Spanners images

1) you need to flatten Johans file structure  
2) Do it on his drive in a new folder called; flatten_folder  
3) Rename each image acording to a new ID; JS + N_i  (JS = Johan Spanner)
4) Draw 5000 random images  
5) Save in new folder: sample_5000  
5) crate csv frame - that is csv with column names and row index  
6) column names = features  
7) row index = image ID  
8) codebook  
9) Page in code book showing the order of the column names so they don't have to scroll 

to mount external harddriv:
> sudo mkdir /mnt/f # f is just random letter for the mounting point  
> sudo mount -t drvfs d: /mnt/f # f is the letter choosen above, d is the drive name

Now you can access his files at location: **/mnt/f/'October Johan Archive'/Spanner**

You then have the folders Published, Raw, Submitted.

# Code

In [1]:
import os
import cv2
from iptcinfo3 import IPTCInfo
from PIL import Image
import re
import zipfile
import shutil
import numpy as np
import time
import matplotlib.pyplot as plt

In [2]:
# external_dir =  "/mnt/f/OctoberJohanArchive"
external_dir =  "/mnt/f/OctoberJohanArchiveTest" # only the published images
flatten_dir =  "/mnt/f/OctoberJohanArchiveFlatten"

In [3]:
def count_files(external_dir):
    
    """Counts files in folder tree. Sanity check, and fits in the function flatten_dirs()"""
    
    count = 0
    
    for root, dirs, files in os.walk(external_dir):
        for name in files:
            count += 1
        
    return(count)

In [4]:
def mk_flatten_dir(flatten_dir):
    
    """Creates a new folder which is to be the location of the flattened folder tree. fits in the function flatten_dirs()"""
    
    if not os.path.exists(flatten_dir):
        os.mkdir(flatten_dir) # makes new dir                     
        
    # Or REPLACE old if exist
    else:
        shutil.rmtree(flatten_dir) # delets dir and content.. FOREVER!
        os.mkdir(flatten_dir) # makes new dir


In [14]:
# def infoIPTC() 
#     """Takes infomation from the image path string, and add it to the IPTC. Ex, wheter the is is raw, edit or puplished"""

In [6]:
def flatten_dirs(external_dir, flatten_dir):
    
    time_start = time.time() # start time
    mk_flatten_dir(flatten_dir) # creates the new folder
    files_count = count_files(external_dir) # counts all files to be moved
    counter = 0 # counter for all iterations
    count_move = 0 # counter for files moved and image ID
    count_exceptions = 0 # count files not being move, due to them not being images.
    ID_front = "JS" # Johan Spanner
    
    print(f'\nstarts moving files from: {external_dir}\nto new shared parent dir: {flatten_dir}...\n')
    
    for root, dirs, files in os.walk(external_dir):
        
        for name in files: 
            
            counter += 1
            print(f'Handling file {counter} of {files_count}', end= '\r')
                        
            old_path = os.path.join(root, name) #all the info for the IPTC should be in in old_path name
            
            #store info for IPTC
            old_path.split("/") # list of strings
            
            try: # you only wnat to move actual images
                
                # verify that is an image
                img = Image.open(old_path)  # open the image file
                img.verify()  # verify that it is a good image, without decoding it.. quite fast
                img.close()
                
                # new path
                count_move += 1
                new_path = flatten_dir + "/" + ID_front + str(count_move) + ".jpeg" # path and image ID - could also put the ID in the IPTC now you are at it.
                # seems you can call all relevant image types .jpeg and still open them - even .tif
                
                # Moving day..
                shutil.copyfile(old_path, new_path) # x6 slower then move.. IPTC moves to
#                 shutil.move(old_path, new_path)# will delete the original path. Do IPTC also move? 
#                 shutil.copy(old_path, new_path) # error for both copy and copy2

            except Exception as e: # Non images you just pass
        
                count_exceptions += 1
                pass
                
    # Logging time it takes to complete operation..
    time_end = time.time()
    time_run = round((time_end - time_start)/60,2)
                    
    print(f'\n\nDone moving files to parent dir: {flatten_dir}...')

    print(f'File(s) moved: {counter-count_exceptions} out of {files_count} files')
    print(f'File(s) not moved (not images): {count_exceptions} out of {files_count} files')
    
    print(f'\nALL DONE!\nThe operation took {time_run} minutes')
    


In [7]:
flatten_dirs(external_dir, flatten_dir)


starts moving files from: /mnt/f/OctoberJohanArchiveTest
to new shared parent dir: /mnt/f/OctoberJohanArchiveFlatten...

Handling file 531 of 531

Done moving files to parent dir: /mnt/f/OctoberJohanArchiveFlatten...
File(s) moved: 524 out of 531 files
File(s) not moved (not images): 7 out of 531 files

ALL DONE!
The operation took 1.1 minutes


In [29]:
test_img = "/mnt/f/OctoberJohanArchive/Spanner/Submitted/2003/031703Jylleditfirstnight/A-edit/DSC_0148.tif"

test_img.split("/")#[5]

['',
 'mnt',
 'f',
 'OctoberJohanArchive',
 'Spanner',
 'Submitted',
 '2003',
 '031703Jylleditfirstnight',
 'A-edit',
 'DSC_0148.tif']

## Check IPTC in moved images.

'date created' = date 
'time created' = time
'copyright notice' = Johan spanner
'custom1' = raw, submitted, pulished

In [11]:
dict_keys = ['object name', 
             'edit status', 
             'editorial update', 
             'urgency', 
             'subject reference', 
             'category', 
             'supplemental category', 
             'fixture identifier', 
             'keywords', 
             'content location code', 
             'content location name', 
             'release date', 
             'release time', 
             'expiration date', 
             'expiration time', 
             'special instructions', 
             'action advised', 
             'reference service', 
             'reference date', 
             'reference number', 
             'date created', 
             'time created', 
             'digital creation date', 
             'digital creation time', 
             'originating program', 
             'program version', 
             'object cycle', 
             'by-line', 
             'by-line title', 
             'city', 
             'sub-location', 
             'province/state', 
             'country/primary location code', 
             'country/primary location name', 
             'original transmission reference', 
             'headline', 
             'credit', 
             'source', 
             'copyright notice', 
             'contact', 
             'caption/abstract', 
             'local caption', 
             'writer/editor', 
             'image type', 
             'image orientation', 
             'language identifier', 
             'custom1', 
             'custom2', 
             'custom3', 
             'custom4', 
             'custom5', 
             'custom6', 
             'custom7', 
             'custom8', 
             'custom9', 
             'custom10', 
             'custom11', 
             'custom12', 
             'custom13', 
             'custom14', 
             'custom15', 
             'custom16', 
             'custom17', 
             'custom18', 
             'custom19', 
             'custom20'] # surely you can get this list from somewhere...
# save as a pickle so you don't have to look at this shit.

In [75]:
def get_IPTC_data(image):
    info = IPTCInfo(image, force=True, inp_charset='utf8')
    
    keys = ['custom1']
    strings = ['submitted']
    
    for i in keys:
        if info[i] == None or len(info[i]) < 1:
                info[i] = strings[0] # if you sort the string list first, you could use enumarate and strings[j]
                
    #--------------------------------------------
    
    for i in dict_keys:
        if info[i] != None:
            if len(info[i]) > 0: 
                print(f'key: {i}, info: {info[i]}\n')


In [76]:
test_img = "/mnt/f/OctoberJohanArchiveTest/Spanner_Green_Zone_9916.jpg"

get_IPTC_data(test_img)

key: date created, info: 20090101

key: time created, info: 113111+0300

key: by-line, info: Johan Spanner

key: city, info: Baghdad

key: country/primary location name, info: Iraq

key: headline, info: Green Zone handover

key: credit, info: Johan Spanner for The New York Times

key: copyright notice, info: Johan Spanner

 he ceremony was attended by Iraqi Minister of Defense, National security, top Iraqi and some American army colonels, Iraqi police Generals and the American Embassy and the head of the Baghdad provincial Council dedicated to the handover of the Green Zone. Iraqi national anthem was played several times. Minister of Defense, Brigadier General Emad, commander of Baghdad Brigade which will control the Green Zone, and Colonel Ferrari who commanded the American unit in the Green Zone, gave speeches.

key: custom1, info: submitted

